In [2]:
%cd drive/MyDrive/Gesture_detection/
!pwd

/content/drive/MyDrive/Gesture_detection
/content/drive/MyDrive/Gesture_detection


In [3]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_df = pd.read_csv('keypoint.csv',header=None)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,0.0,0.0,0.200787,-0.051181,0.366142,-0.181102,0.484252,-0.307087,0.594488,...,0.039370,-0.976378,-0.074803,-0.429134,-0.145669,-0.578740,-0.185039,-0.685039,-0.220472,-0.787402
1,0,0.0,0.0,0.206349,-0.043651,0.376984,-0.162698,0.507937,-0.273810,0.615079,...,0.047619,-0.980159,-0.079365,-0.428571,-0.142857,-0.583333,-0.182540,-0.690476,-0.210317,-0.793651
2,0,0.0,0.0,0.202381,-0.043651,0.373016,-0.162698,0.500000,-0.277778,0.607143,...,0.039683,-0.980159,-0.083333,-0.428571,-0.146825,-0.579365,-0.186508,-0.686508,-0.214286,-0.789683
3,0,0.0,0.0,0.207171,-0.039841,0.382470,-0.159363,0.509960,-0.274900,0.617530,...,0.051793,-0.980080,-0.079681,-0.426295,-0.143426,-0.581673,-0.183267,-0.689243,-0.211155,-0.792829
4,0,0.0,0.0,0.206349,-0.039683,0.380952,-0.162698,0.507937,-0.281746,0.615079,...,0.051587,-0.980159,-0.079365,-0.428571,-0.142857,-0.583333,-0.178571,-0.690476,-0.210317,-0.793651


In [4]:
num_classes = 4
y = train_df[0].values
X_data = train_df.drop(0,axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X_data, y, train_size=0.85, random_state=42)
val_size = int(0.1 * len(X_train))

X_train = X_train.iloc[val_size:]
y_train = y_train[val_size:]

# transform to tensors
X_val = torch.tensor(X_train.iloc[:val_size].values)
y_val = torch.tensor(y_train[:val_size])

X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test.values)
y_test = torch.tensor(y_test)


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(len(y_train))
print(y_val[0])
print(y_test.size())




torch.Size([3662, 42])
torch.Size([406, 42])
torch.Size([719, 42])
3662
tensor(2)
torch.Size([719])


In [ ]:
print(len(y))

128


In [14]:
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=128,pin_memory=True)
val_loader = DataLoader(TensorDataset(X_val,y_val), batch_size=64,pin_memory=True)

In [13]:
for x,y in train_loader:
  print(x[0])
  print(y[0])
  print(x.shape)
  print(y.shape)
  print(len(train_loader))
  break

tensor([ 0.0000,  0.0000,  0.0373, -0.2241, -0.0124, -0.4066, -0.1037, -0.5436,
        -0.1992, -0.6141, -0.0083, -0.4896, -0.1079, -0.7510, -0.1701, -0.8963,
        -0.2075, -1.0000, -0.1245, -0.4232, -0.2531, -0.6058, -0.2241, -0.5436,
        -0.1701, -0.4813, -0.2282, -0.3402, -0.3112, -0.4772, -0.2656, -0.4232,
        -0.2199, -0.3693, -0.3154, -0.2531, -0.3444, -0.3651, -0.2946, -0.3195,
        -0.2614, -0.2656], dtype=torch.float64)
tensor(2)
torch.Size([128, 42])
torch.Size([128])
29


In [7]:
model = nn.Sequential(
    nn.Linear(42,20),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(20,10),
    nn.ReLU(),
    nn.Linear(10,num_classes)
)
model

Sequential(
  (0): Linear(in_features=42, out_features=20, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=20, out_features=10, bias=True)
  (4): ReLU()
  (5): Linear(in_features=10, out_features=4, bias=True)
)

In [8]:
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)
model = model.to(device)

In [11]:
class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [13]:
def train(train_loader,model,epoch,criterion):
  accs = AverageMeter()
  losses = AverageMeter()
  model.train()
  for i, (x,y) in enumerate(train_loader):

    x = x.to(device)
    y = y.to(device)
    scores = model(x.float())
    loss = criterion(scores,y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.update(loss)
    predicted = torch.argmax(scores,dim=1)
    correct = (predicted == y).sum()
    acc = correct / len(y)
    accs.update(acc)

    # if i % 100 == 0:
    print(f'Epoch: [{epoch}] [{i}/{len(train_loader)}]  Loss: {losses.avg:.4f}  Acc: {accs.avg * 100:.3f}%')
    
  

In [14]:
def validate(val_loader,model,epoch,criterion):
  accs = AverageMeter()
  losses = AverageMeter()
  model.eval()
  with torch.no_grad():
    for i, (x,y) in enumerate(val_loader):

      x = x.to(device)
      y = y.to(device)
      scores = model(x.float())
      loss = criterion(scores,y)      

      losses.update(loss)
      predicted = torch.argmax(scores,dim=1)
      correct = (predicted == y).sum()
      acc = correct / len(y)
      accs.update(acc)

    # if i % 100 == 0:
      print(f'Validation: [{i}/{len(val_loader)}]  Loss: {losses.avg:.4f}  Acc: {accs.avg * 100:.3f}%')

In [23]:
epochs=100

for e in range(epochs):
  train(train_loader,model,e,criterion)
  validate(val_loader,model,e,criterion)
  torch.save(model.state_dict(), f'checkpoint_gesture.pt')



Epoch: [0] [0/29]  Loss: 0.0364  Acc: 98.438%
Epoch: [0] [1/29]  Loss: 0.0908  Acc: 96.875%
Epoch: [0] [2/29]  Loss: 0.1049  Acc: 96.615%
Epoch: [0] [3/29]  Loss: 0.0860  Acc: 97.070%
Epoch: [0] [4/29]  Loss: 0.0754  Acc: 97.500%
Epoch: [0] [5/29]  Loss: 0.0712  Acc: 97.656%
Epoch: [0] [6/29]  Loss: 0.0743  Acc: 97.656%
Epoch: [0] [7/29]  Loss: 0.0695  Acc: 97.754%
Epoch: [0] [8/29]  Loss: 0.0739  Acc: 97.569%
Epoch: [0] [9/29]  Loss: 0.0747  Acc: 97.500%
Epoch: [0] [10/29]  Loss: 0.0746  Acc: 97.656%
Epoch: [0] [11/29]  Loss: 0.0712  Acc: 97.786%
Epoch: [0] [12/29]  Loss: 0.0738  Acc: 97.716%
Epoch: [0] [13/29]  Loss: 0.0709  Acc: 97.768%
Epoch: [0] [14/29]  Loss: 0.0717  Acc: 97.656%
Epoch: [0] [15/29]  Loss: 0.0698  Acc: 97.754%
Epoch: [0] [16/29]  Loss: 0.0714  Acc: 97.748%
Epoch: [0] [17/29]  Loss: 0.0732  Acc: 97.656%
Epoch: [0] [18/29]  Loss: 0.0728  Acc: 97.656%
Epoch: [0] [19/29]  Loss: 0.0730  Acc: 97.578%
Epoch: [0] [20/29]  Loss: 0.0747  Acc: 97.507%
Epoch: [0] [21/29]  Los

In [27]:
model = nn.Sequential(
    nn.Linear(42,20),
    nn.ReLU(),
    nn.Dropout(p=0.4),
    nn.Linear(20,10),
    nn.ReLU(),
    nn.Linear(10,num_classes)
)
model.load_state_dict(torch.load('checkpoint_gesture999.pt'))
test_loader = DataLoader(TensorDataset(X_test,y_test), batch_size=len(y_test),pin_memory=True)
accs = AverageMeter()
model.eval()
with torch.no_grad():
  for i, (x,y) in enumerate(val_loader):

    x = x
    y = y
    scores = model(x.float())

    predicted = torch.argmax(scores,dim=1)
    correct = (predicted == y).sum()
    acc = correct / len(y)
    accs.update(acc)
print(f'Accuracy on test data: {accs.avg * 100:.3f}%')


Accuracy on test data: 99.554%


**Point history classification**

In [4]:
num_classes = 4
time_steps = 16
dimension = 2

In [5]:
train_df = pd.read_csv('point_history.csv',header=None)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0,0,0,-0.002083,-0.003704,-0.002083,-0.003704,-0.003125,-0.005556,-0.002083,...,-0.003125,-0.016667,-0.002083,-0.018519,-0.002083,-0.016667,-0.001042,-0.016667,0.000000,-0.014815
1,0,0,0,0.001042,0.000000,0.002083,0.000000,0.001042,0.001852,0.002083,...,-0.001042,0.003704,-0.001042,0.003704,-0.001042,0.001852,-0.002083,0.005556,-0.002083,0.005556
2,0,0,0,0.001042,0.000000,0.000000,0.001852,0.001042,0.001852,0.001042,...,-0.002083,0.003704,-0.002083,0.001852,-0.003125,0.005556,-0.003125,0.005556,-0.002083,0.007407
3,0,0,0,-0.001042,0.001852,0.000000,0.001852,0.000000,0.001852,0.000000,...,-0.003125,0.001852,-0.004167,0.005556,-0.004167,0.005556,-0.003125,0.007407,-0.004167,0.007407
4,0,0,0,0.001042,0.000000,0.001042,0.000000,0.001042,0.000000,0.002083,...,-0.003125,0.003704,-0.003125,0.003704,-0.002083,0.005556,-0.003125,0.005556,-0.002083,0.005556


In [6]:
y = train_df[0].values
X_data = train_df.drop(0,axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X_data, y, train_size=0.85, random_state=42)
val_size = int(0.1 * len(X_train))

X_train = X_train.iloc[val_size:]
y_train = y_train[val_size:]

# transform to tensors
X_val = torch.tensor(X_train.iloc[:val_size].values)
y_val = torch.tensor(y_train[:val_size])

X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test.values)
y_test = torch.tensor(y_test)


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(len(y_train))
print(y_val[0])
print(y_test.size())

torch.Size([4051, 32])
torch.Size([450, 32])
torch.Size([795, 32])
4051
tensor(0)
torch.Size([795])


In [7]:
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=128,pin_memory=True)
val_loader = DataLoader(TensorDataset(X_val,y_val), batch_size=64,pin_memory=True)

In [36]:
class PointHistoryClassifier(nn.Module):

  def __init__(self,num_classes = 4,time_steps = 16,hidden_size=64):
    super().__init__()
    self.dropout = nn.Dropout(p=0.5)
    self.LSTM = nn.LSTM(2,hidden_size,batch_first=True)
    self.linear1 = nn.Linear(hidden_size*time_steps,10)
    self.linear2 = nn.Linear(10,num_classes)
    self.relu = nn.ReLU()
    self.time_steps = time_steps
  def forward(self,x,states):
    
    x = x.reshape(x.size(0),self.time_steps,2)
    out , (h,c) = self.LSTM(x,states) # out - (batch_size,time_steps,hidden_size) ; h - (1,batch_size,hidden_size)
    out = self.dropout(out.reshape(out.size(0),-1))
    out = self.relu(self.linear1(out))
    out = self.linear2(out) # (batch_size,num_classes)

    return out


# model = PointHistoryClassifier()
# x = torch.ones(1,time_steps*dimension)
# states = (torch.zeros(1, 1, 64),
#               torch.zeros(1,1,64))


In [37]:
model = PointHistoryClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

In [32]:
def train(train_loader,model,epoch,criterion):
  accs = AverageMeter()
  losses = AverageMeter()
  model.train()
  for i, (x,y) in enumerate(train_loader):

    states = (torch.randn(1, x.size(0), 64).to(device),
              torch.randn(1,x.size(0), 64).to(device))
    x = x.to(device)
    y = y.to(device)
    scores = model(x.float(),states)
    loss = criterion(scores,y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.update(loss)
    predicted = torch.argmax(scores,dim=1)
    correct = (predicted == y).sum()
    acc = correct / len(y)
    accs.update(acc)

    if i % 10 == 0:
      print(f'Epoch: [{epoch}] [{i}/{len(train_loader)}]  Loss: {losses.avg:.4f}  Acc: {accs.avg * 100:.3f}%')
    
  

In [33]:
def validate(val_loader,model,epoch,criterion):
  accs = AverageMeter()
  losses = AverageMeter()
  model.eval()
  with torch.no_grad():
    for i, (x,y) in enumerate(val_loader):

      states = (torch.randn(1, x.size(0), 64).to(device),
              torch.randn(1,x.size(0), 64).to(device))
      x = x.to(device)
      y = y.to(device)
      scores = model(x.float(),states)
      loss = criterion(scores,y)      

      losses.update(loss)
      predicted = torch.argmax(scores,dim=1)
      correct = (predicted == y).sum()
      acc = correct / len(y)
      accs.update(acc)

      if i % 4 == 0:
        print(f'Validation: [{i}/{len(val_loader)}]  Loss: {losses.avg:.4f}  Acc: {accs.avg * 100:.3f}%')

In [38]:
epochs=300

for e in range(epochs):
  train(train_loader,model,e,criterion)
  validate(val_loader,model,e,criterion)
  torch.save(model.state_dict(), 'checkpoint_pointHistory.pt')

Epoch: [0] [0/32]  Loss: 1.4064  Acc: 30.469%
Epoch: [0] [10/32]  Loss: 1.3937  Acc: 24.361%
Epoch: [0] [20/32]  Loss: 1.3896  Acc: 25.930%
Epoch: [0] [30/32]  Loss: 1.3891  Acc: 25.983%
Validation: [0/8]  Loss: 1.3864  Acc: 26.562%
Validation: [4/8]  Loss: 1.3801  Acc: 30.312%
Epoch: [1] [0/32]  Loss: 1.3712  Acc: 36.719%
Epoch: [1] [10/32]  Loss: 1.3860  Acc: 26.776%
Epoch: [1] [20/32]  Loss: 1.3850  Acc: 27.307%
Epoch: [1] [30/32]  Loss: 1.3856  Acc: 26.865%
Validation: [0/8]  Loss: 1.3865  Acc: 26.562%
Validation: [4/8]  Loss: 1.3802  Acc: 30.000%
Epoch: [2] [0/32]  Loss: 1.3719  Acc: 38.281%
Epoch: [2] [10/32]  Loss: 1.3844  Acc: 26.847%
Epoch: [2] [20/32]  Loss: 1.3841  Acc: 27.083%
Epoch: [2] [30/32]  Loss: 1.3857  Acc: 26.915%
Validation: [0/8]  Loss: 1.3818  Acc: 26.562%
Validation: [4/8]  Loss: 1.3813  Acc: 29.375%
Epoch: [3] [0/32]  Loss: 1.3706  Acc: 37.500%
Epoch: [3] [10/32]  Loss: 1.3864  Acc: 27.557%
Epoch: [3] [20/32]  Loss: 1.3856  Acc: 27.381%
Epoch: [3] [30/32]  Los